In [1]:
import regression
import tests.utils
import numpy as np

In [2]:
x, y, y_true, reference_model = tests.utils.data_generation_model(100, 3)

In [3]:
verify = reference_model.data_matrix(x)

In [4]:
verify

array([[1.        , 0.        , 0.        ],
       [1.        , 2.55434788, 4.79669863],
       [1.        , 0.83972714, 1.50958104],
       [1.        , 0.        , 0.        ],
       [1.        , 1.83464013, 0.72784216],
       [1.        , 1.06496364, 1.72181839],
       [1.        , 1.19548782, 0.        ],
       [1.        , 0.47810478, 0.63047479],
       [1.        , 1.76125324, 1.73398255],
       [1.        , 0.        , 0.        ],
       [1.        , 1.81904615, 0.        ],
       [1.        , 1.61196675, 4.81058135],
       [1.        , 1.81956843, 2.86311467],
       [1.        , 2.30620946, 0.27536615],
       [1.        , 1.06122468, 0.83097163],
       [1.        , 0.        , 0.        ],
       [1.        , 1.65856421, 2.43795471],
       [1.        , 0.88135707, 2.62770755],
       [1.        , 3.67935169, 9.21240092],
       [1.        , 1.03039234, 1.84105898],
       [1.        , 0.76028982, 0.        ],
       [1.        , 1.08706175, 1.76898612],
       [1.

In [5]:
v = np.zeros((10, 10), dtype=int)
t = np.zeros((10, 10))
hinge = v > 1
where = v > 1

In [6]:
for i, basis in enumerate(reference_model.basis):
    v[:len(basis.v), i] = basis.v
    t[:len(basis.t), i] = basis.t
    hinge[:len(basis.hinge), i] = basis.hinge
    where[:len(basis.hinge), i] = True

In [7]:
result = -t + x[:, v] 
np.maximum(np.zeros_like(result), result, where=hinge, out=result)
result = result.prod(axis=1, where=where)[:, :]
result

array([[1.        , 0.        , 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 2.55434788, 4.79669863, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 0.83972714, 1.50958104, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 0.        , 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.83464013, 0.72784216, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.06496364, 1.72181839, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.19548782, 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 0.47810478, 0.6304747

In [13]:
print(where[:1])

[[False  True  True False False False False False False False]]


In [17]:
print(t)

[[0.         0.9851656  0.9851656  0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.79332617 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.

In [16]:
reference_model.basis

[Basis(v=[], t=array([], dtype=float64), hinge=[]),
 Basis(v=[0], t=array([0.9851656]), hinge=[True]),
 Basis(v=[0, 1], t=array([0.9851656 , 0.79332617]), hinge=[True, True])]